In [14]:
pip install fastapi

In [15]:
pip install uvicorn

In [ ]:
from typing import Optional 
from fastapi import FastAPI 

app = FastAPI() 

@app.get("/") 
def read_root(): 
  return {"Hello": "World"} 
  
@app.get("/items/{item_id}") 
def read_item(item_id: int, q: Optional[str] = None): 
  return {"item_id": item_id, "q": q}


In [ ]:
from fastapi import FastAPI 
from time import time 
import asyncio 

app = FastAPI() 

URL = "https://www.pexels.com/ko-kr/search/videos/%EC%96%BC%EA%B5%B4/" 

async def request(client): 
  response = await client.get(URL) 
  return response.text 
  
async def task(): 
  async with httpx.AsyncClient() as client: 
    tasks = [request(client) for i in range(100)] 
    result = await asyncio.gather(*tasks) 
    print(result) 
    
    
@app.get('/') 
async def f(): 
  start = time() 
  await task() 
  print("time: ", time() - start)


In [ ]:
!ngrok authtoken 26PXxhhXNfcMTG7wcAUGx5qr0rn_2xQ3qGzbtjR7BDpmEFqT3

/bin/bash: ngrok: command not found


### PastAPI 더빙

In [19]:
pip install pyngrok

     |████████████████████████████████| 745 kB 5.2 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=44bcf88fb2c5f8a2ed0f60b2f7a7d8cbe3aa7faa63bbf7164c859c61d84c1556
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [26]:
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse
from pyngrok import ngrok
import nest_asyncio
from typing import Optional, List
import uvicorn
from fastapi import File, UploadFile
import os
import numpy as np

import urllib.request
import requests
import json


app = FastAPI()



@app.get('/')
def read_root():
  return {'hello':'world'}


class ClovaSpeechClient:
  # Clova Speech invoke URL
  invoke_url = 'https://clovaspeech-gw.ncloud.com/external/v1/2389/135ddb361331ff2821f583d720d636d5e9fc782ab76ece949d5ab49b83e39c85'
  # Clova Speech secret key
  secret = 'KdUtICuxadSxcSYRrM2EBHjIyC5mJMsnNtDsxFlE'

  def req_upload(self, file, completion, callback=None, userdata=None, forbiddens=None, boostings=None,
                  wordAlignment=True, fullText=True, diarization=None):
      request_body = {
          'language': 'enko',
          'completion': completion,
          'callback': callback,
          'userdata': userdata,
          'wordAlignment': wordAlignment,
          'fullText': fullText,
          'forbiddens': forbiddens,
          'boostings': boostings,
          'diarization': diarization,
      }
      headers = {
          'Accept': 'application/json;UTF-8',
          'X-CLOVASPEECH-API-KEY': self.secret
      }
      print(json.dumps(request_body, ensure_ascii=False).encode('UTF-8'))
      files = {
          'media': open(file, 'rb'),
          'params': (None, json.dumps(request_body, ensure_ascii=False).encode('UTF-8'), 'application/json')
      }
      response = requests.post(headers=headers, url=self.invoke_url + '/recognizer/upload', files=files)
      return response

@app.post('/speech/text')
async def speech_text(files: List[UploadFile] = File(...)):
  for file in files:
    contents = await file.read()
    with open(os.path.join('/content/',file.filename), 'wb') as fp:
      fp.write(contents)
    if __name__ == '__main__':
    # res = ClovaSpeechClient().req_url(url='https://www.youtube.com/watch?v=035BEi1zXiY', completion='sync')
    # res = ClovaSpeechClient().req_object_storage(data_key='data/media.mp3', completion='sync')
      res = ClovaSpeechClient().req_upload(file=(os.path.join('/content/' + file.filename)), completion='sync')
      # res = ClovaSpeechClient().req_upload(file='data/media.mp3', completion='sync')

    result = res.json()['text']

    # translation
    client_id = "b13psmxh1d"
    client_secret = "KdUtICuxadSxcSYRrM2EBHjIyC5mJMsnNtDsxFlE"
    encText = urllib.parse.quote(result)
    data = "source=en&target=ko&honorific=True&text=" + encText  # 영어 한국어 설정
    url = "https://naveropenapi.apigw.ntruss.com/nmt/v1/translation"
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
    request.add_header("X-NCP-APIGW-API-KEY",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        print(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)
        

    mas = response_body.decode('utf-8')
    translation = mas.split('"')[-2]

    # voice
    encText = urllib.parse.quote(translation)
    data = "speaker=nara&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
    url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
    request.add_header("X-NCP-APIGW-API-KEY",client_secret)
    response = urllib.request.urlopen(request, data=data.encode('utf-8'))
    rescode = response.getcode()
    if(rescode==200):
        # print("TTS mp3 저장")
        response_body = response.read()
        with open('./(%s)voice.mp3'% file.filename, 'wb') as f:
            f.write(response_body)
    else:
        print("Error Code:" + rescode)
  
  return HTMLResponse(content=F"<h1>{'저장완료'}", status_code=200)

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:' , ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, host='0.0.0.0', port=8000)

Public URL: http://f31f-35-237-195-1.ngrok.io


INFO:     Started server process [59]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [59]


In [24]:
!ngrok authtoken 26PXxhhXNfcMTG7wcAUGx5qr0rn_2xQ3qGzbtjR7BDpmEFqT3

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [21]:
pip install python-multipart

  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=22af97527ed19d38e522faa1b75fb44cd8c823c25629de5246674b87e7cafb9d
  Stored in directory: /root/.cache/pip/wheels/2c/41/7c/bfd1c180534ffdcc0972f78c5758f89881602175d48a8bcd2c
Successfully built python-multipart
